<a href="https://colab.research.google.com/github/DarekGit/FACES_DNN/blob/master/notebooks/04_04_MOBILENETV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Spis Treści](https://github.com/DarekGit/FACES_DNN/blob/master/notebooks/Praca_Dyplomowa.ipynb)<br>
[4. Metody detekcji](04_00_Modele.ipynb)


##MobileNet version 2 
W 2017 roku Zespół Google z dużym sukcesem wprowadził MobileNet V1 do klasyfikacji obrazów jak i ekstracji cech w większych sieciach neuronowych.

<br> Już rok później w 2018 zespół AI Google <a href="https://research.googleblog.com/2018/04/mobilenetv2-next-generation-of-on.html"> powiadomił o wprowadzeniu MobileNet version 2</a>. <br>Na bazie V1 udało sie stworzyć jeszcze bardziej efektywną i wydajną wersję, **do zostosowania na urządzeniach mobilnych.**


###<br>Główne cechy V1
Podstawowym pomysłem w V1 było wykorzystanie sperowalnych sieci konwolucyjnych w celu obniżenie zapotrzebowania na moc obliczniową. Standardowe sieci konwolucyjne wymagają wykonania dużej liczby obliczeń, których ilość jest proporcjonalny do iloczynu ilośc kanałów wejściowych, ilości kanałów wyjściowych, wielkości danych wejściowych i wielkości kernela. Zastosowanie konwolucji separowalnych po kanałach, obniża zapotrzebowanie na moc obliczeniową klikukrotnie.

Podstawowa warstwa konwolucyjna została podzielona na dwa etapy: pierwszy to warstwa <a href="https://machinethink.net/blog/googles-mobile-net-architecture-on-iphone/">konwolucji separowanej po kanałach (depthwise convolution)</a>, po której mamy warstwę konwolucji 1×1 (pointwise) łączącej cechy z kanałów:

<div align="center"><br>
<img src="https://github.com/DarekGit/FACES_DNN/blob/master/Figures/MN2/MN2_SeparableConv.png" alt="A depthwise separable convolution block" width="600" ><br>
</div>

<br>Razem obie warstwy tworzą blok konwolucyjny, który wykonuje podobne operacje do standardowej sieci konwolucyjnej, ale znacznie szybciej. (podobne rozwiązania stosujemy w sieciach typu Inception).
<br>Architektura Mobilenet V1 składa się z standardowej warstwy konwolucyjnej 3x3 a następnie z trzynastu bloków opisanych powyżej.

Pomiędzy warstawami konwolucji separowalnych nie stosujemy warstw Pool, zamist tego wybrane warsty konwolucji mają stride równe 2 w celu redukcji rozmiaru danych. Przy zastosowaniu redukcji rozmiaru danych, jednocześnie dublowana jest ilość kanałów wyjściowych w warstwie konwolucji 1x1 (pointwise). Przykładowo dla obrazu wejściowego o rozmiarze 224×224×3 na wyjściu sieci otrzymujemy mapę cech o rozmiarze 7×7×1024.

Tak jak w większości współczesnych archiotektur, po warstwach konwolucyjnych stosujemy batch normalization, a jako funkcję aktywacji **ReLU6**. ReLU6 ma dodatkowe ograniczenie aby uniknąć zbyt dużych wartości:

<pre><code class="language-python">y = min(max(0, x), 6)
</code></pre>

Autorzy MobileNet w dokumentacji wskazują, iż sieć z ReLU6 jest bardziej odporna niż ReLU dla obliczeń z małą precyzją (16-bit float np iOS). 

Kształt ReLU6 ma zbiżone cechy do funckji sigmoid:

<div align="center"><br>
<img src="https://github.com/DarekGit/FACES_DNN/blob/master/Figures/MN2/MN2_ReLU6.png" alt="The ReLU6 activation function" width="600" ><br>
</div>

<br>Sieć MobileNet w zastosowaniach do klasyfikacji kończy się blokiem składającycm się z warstwy Average Pooling, FC lub alternatywnie Convolution 1x1 oraz Softmax.

<br>Sieć Mobilenet została przygotowana jako rodziana sieci. Mamy kilka hiperparametrów umożliwiających stosowanie różnych wersji sieci Mobilenet.
<br>Najważniejszym hiperparametrem jest  <strong>depth multiplier</strong>, czasami nazywany "width multiplier". Zmienia on ilość kanałów w każdej warstwie. 
<br>Stosując depth multiplier równy 0.5 obniżamy ilość kanałów, a tym samym ilość koniecznych obliczeń czterokrotnie oraz ilość parametrów sieci trzykrotnie, dzięki czemy sieć jest szybsza ale  mniej dokładna.

<br>Dzięki zasosowaniu bloków z konwolucją separowalną MobileNet potrzebuje 9-krotnie mniej mocy obliczeniowej do porównywalnych sieci neuronowych z tą samą dokłądnością klasyfikacji. Taki typ sieci może być stosowany z powodzeniem w czasie rzeczywistym do 200 warstw na iPhone 6s.

<br>Więcej szczegółów dostępne jest na stronie: https://arxiv.org/pdf/1704.04861.pdf

###<br>Wersja 2 MobileNet

<p><a href="https://arxiv.org/pdf/1801.04381.pdf">MobileNet V2</a> bazuje na V1 i używa separowalnych warstw konwolucyjnych, a jego podstawowy blok wygląda następująco:

<div align="center"><br>
<img src="https://github.com/DarekGit/FACES_DNN/blob/master/Figures/MN2/MN2_ResidualBlock.png" alt="The bottleneck residual block" width="600" ><br>
</div>

<br>W V2 mamy trzy warstwy konwolucyjne w bloku, dwie ostatnie są zgodne z wersją V1, ale konwolucja 1x1 ma inne zadanie. W wersji V1 zadaniem konwolucji 1x1 (pointwise) było utrzymanie lub zdublowanie ilości kanałów. W przypadku wersji V2 jej zadaniem jest zmniejszenie ilości kanałów. Dlatego nazywana jest **projection layer**, odwzorowuje większą ilość kanałów wejsciowych na tensor z mniejszą ilością kanałów.
<br>Przykładowo warstwa Depthwise może pracowanć ze 144 kanałami, które są ograniczane do tylko 24 w warstwie Pointwise. Czasami warstwę Pointwise nazywa się **bottlnect layer** poniewż zawęża ilośc danych przepływających przez sieć.

<br> Pierwsza warstwa w bloku spełnia nowe zadanie. Jest to również warstwa konwolucyjna 1x1, a jej zadaniem jest rozszerzenie ilości kanałów danych przed warstwą DepthWise. Nazywa się ją **expansion layer** i zawsze zwiększa ilość kanałów (odwrotnie niż **projection layer**).
Współczynnik ekspansji podawany jest jako hiperparametr i można go zmieniać. Domyślnie przyjmowany jest jako 6 (**expansion factor**).
<br>Pryzkładowo dla 24 kanałów wejściowych do bloku, Expansion Layer powiększa ilość kanałów do 144, następnie wykonywana jest konwolucja separowalna (Depthwise) na 144 kanałach i ostatecznie Projection Layer redukuje ilość kanałów do 24.

<div align="center"><br>
<img src="https://github.com/DarekGit/FACES_DNN/blob/master/Figures/MN2/MN2_Expand.png" alt="Expansion and projection" width="600" ><br>
</div>

<br>Tak więc wejscie i wyjscie ma mniejszy rozmiar, podczas gdy rozmiar wewnątrz bloku jest większy.
<br><br>Drugą nową cechą MobileNetV2 jest **residual connection**. Działa ono podobnie do ResNet i pomaga w utrzymaniu przepływu gradientu w sieci. Residual connection używane jest tylko i wyłądcznie w przypadku kiedy ilość kanałów wejściowych jest równa ilości kanałów wyjściowych dla bloku.

<br>Jak w innych sieciach konwolucyjnych każda warstwa ma batch normalization i funkcję aktywacji (ReLU6), oprócz warstwy wyjściowej. Autorzy MobileNet w dokumentacji wskazują, iż warstwa nieliniowa na wyjściu dla małych rozmiarów danych wyjściowych pogarsza wyniki pracy sieci.

<br> Pełna architektura MobileNetV2 składa się ze standardowej warstwy konwolucyjnej 3x3 z 32 kanałami, następnie 17  bloków, zakończonych standardową warstwą konwolucyjną 1x1, Average Pool i warstwą klasyfikacyjną.


###Cel zmian

<br>Pomysł dotyczący V1 opierał się na zmianie złożonej warstwy konwolucyjnej na mniej wymagająca warstwę separowalną **DeepWise**, co okazało sie dużym sukcesem. 
<br>Główną zmianą w wersji V2 jest zastosowanie **residual connection** oraz **expand/projection layers**, co umożliwia zachowanie małego rozmiaru na wejściu i wyjściu bloków:

<div align="center"><br>
<img src="https://github.com/DarekGit/FACES_DNN/blob/master/Figures/MN2/MN2%20_LowDim.png" alt="The tensor dimensions between the blocks" width="600" ><br>
</div>

<br> Rozmiar tensora wyściowego dla wersji V1 (7×7×1024) jest większy od V2. Dzięki zachowaniu małego rozmiaru tensora uzyskano redukcją koniecznej liczby obliczeń dla sieci przy zachowaniu zadawalającej jakości klasyfikacji sieci.
<br>Podaje się porównanie, że mały rozmiar danych pomiędzy blokami odzwierciedla  skompresowane dane rzeczywiste, które są dekompresowane wewnątrz bloków:

<div align="center"><br>
<img src="https://github.com/DarekGit/FACES_DNN/blob/master/Figures/MN2/MN2_Compression.png" alt="Decompression and compression inside the bottleneck residual block" width="600" ><br>
</div>

<br> Expansion Layer działa jako dekompresor, który odtwarza dane rzeczywiste do przetwarzania wewnątrz bloku, a które są następnie kompresowane w Projection Layer w celu zmniejszenia rozmiaru danych na wyjściu bloku. Parametry całego procesu w tym kompresji i dekompresji podlegają optymalizacji w trakcie uczenia sieci.


###<br>Porówanie wersji MobileNet.

<br>Porównanie MobileNet V1 to V2, wymagana ilość obliczeń oraz rozmiar modelu - ilość parametrów:

<table>
<thead>
<tr>
<th>Version</th>
<th>MACs (millions)</th>
<th>Parameters (millions)</th>
</tr>
</thead>

<tbody>
<tr>
<td>MobileNet V1</td>
<td>569</td>
<td>4.24</td>
</tr>

<tr>
<td>MobileNet V2</td>
<td>300</td>
<td>3.47</td>
</tr>
</tbody>
</table>

<br>Wykorzystano dane z <a href="https://github.com/tensorflow/models/tree/master/research/slim/nets/mobilenet_v1.md">Github MNV1</a> oraz  <a href="https://github.com/tensorflow/models/tree/master/research/slim/nets/mobilenet">Gothub MNV2</a>. Modele w wersji z depth multiplier równe 1.0. Dla podanej tabeli, mniejsze liczby oznaczają lepszy wynik.

"MACs - multiply-accumulate operations". Pokazuje ile obliczeń jest wymaganych dla wykonania operacji dla sieci na pojedyńczym obrazie RGB o wymiarach 224x224.
V2 ma dwie przewagi: 
<br>1. potrzebuje prawie dwa razy mniejszą ilość obliczeń.
<br>2. ma mniej parametrów (80%) dzięki czemu szybciej ładuje je z pamięci, co jest istotne dla urządzeń mobilnych, gdzie dostęp do pamięci jest znacznie wolniejszy niż obliczenia.

<br>Poniższa tabela pokazuje maksymalny FPS (frames-per-second) dla różnych urządzeń dla wersji modeli jak wyżej:</p>

<table>
<thead>
<tr>
<th>Version</th>
<th>iPhone 7</th>
<th>iPhone X</th>
<th>iPad Pro 10.5</th>
</tr>
</thead>

<tbody>
<tr>
<td>MobileNet V1</td>
<td>118</td>
<td>162</td>
<td>204</td>
</tr>

<tr>
<td>MobileNet V2</td>
<td>145</td>
<td>233</td>
<td>220</td>
</tr>
</tbody>
</table>

<br> Wyniki uzyskano przy zastosowaniu optymalizacji polegającej na równoczesnym przygtowaniu danych wejściowych na CPU w trakcie przetwarzania danych na GPU.

<br>Porównanie jakości pracy modeli:

<table>
<thead>
<tr>
<th>Version</th>
<th>Top-1 Accuracy</th>
<th>Top-5 Accuracy</th>
</tr>
</thead>

<tbody>
<tr>
<td>MobileNet V1</td>
<td>70.9</td>
<td>89.9</td>
</tr>

<tr>
<td>MobileNet V2</td>
<td>71.8</td>
<td>91.0</td>
</tr>
</tbody>
</table>

<br> Więcej wyników dostępne jest na stronie: http://www.image-net.org/challenges/LSVRC/2012/



##<br>WNIOSKI:
<br>**MobileNet V2 dla wszyskich miar mam lepsze wynik od MobileNet V1. Zarówno jest szybszy jak również ma lepszą dokładaność dla klasyfikacji.**
<br> W innych pracach wskazano, że może być również stosowany do detekcji obiektów i segmentacji obrazów.

<br>Dla klasyfikacji ostatni blok wygląda jak na rysunku poniżej:

<div align="center"><br>
<img src="https://github.com/DarekGit/FACES_DNN/blob/master/Figures/MN2/MN2_Classifier.png" alt="Classifier head on top of the base network" width="600" >
</div>


<br>Użycie MobileNet w SSDlite do detekcji, może wyglądać następująco:


<div align="center"><br>
<img src="https://github.com/DarekGit/FACES_DNN/blob/master/Figures/MN2/MN2_FeatureExtractor.png" alt="Using MobileNet as a feature extractor in a larger network" width="600" ><br>
</div>

<br>W powyższym rozwiązaniu wykorzystano nie tylko cechy z ostaniej warstwy MobileNet, ale również kilka wcześniejszych. Na pokazynym przykładzie MobileNet wykorzystwany jest jako ekstraktor cech obrazu.

<br>**W dalszej części pracy zastosowano MobileNet V2 w detekcji twarzy w modelu Detectron2 z bardzo dobrym wynikiem.**



https://analyticsindiamag.com/why-googles-mobilenetv2-is-a-revolutionary-next-gen-on-device-computer-vision-network/

https://ai.googleblog.com/2018/04/mobilenetv2-next-generation-of-on.html

Recently, a group of researchers from Google released a neural network architecture MobileNetV2, which is optimised for mobile devices. The architecture delivers high accuracy results while keeping the parameters and mathematical operations as low as possible to bring deep neural networks to mobile devices.

Last year, the company introduced MobileNetV1 for Tensorflow, designed to support classification, detection, embedding and segmentation. “The ability to run deep networks on personal mobile devices improves user experience, offering anytime, anywhere access, with additional benefits for security, privacy, and energy consumption. As new applications emerge allowing users to interact with the real world in real time, so does the need for ever more efficient neural networks,” Google researchers Mark Sandler and Andrew Howard said in their research blog post.


The new mobile architecture, MobileNetV2 is the improved version of MobileNetV1 and is released as a part of TensorFlow-Slim Image Classification Library. Developers can even access it in Colaboratory or can download the notebook and explore it using Jupyter. It is also available as modules on TensorFlow-Hub. The pretrained checkpoints can be found on the open source platform GitHub.

What Is MobileNetV2?
MobileNets are small, low-latency, low-power models parameterised to meet the resource constraints of a variety of use cases. According to the research paper, MobileNetV2 improves the state-of-the-art performance of mobile models on multiple tasks and benchmarks as well as across a spectrum of different model sizes. It is a very effective feature extractor for object detection and segmentation. For instance, for detection, when paired with Single Shot Detector Lite, MobileNetV2 is about 35 percent faster with the same accuracy than MobileNetV1.

It builds upon the ideas from MobileNetV1, using depth-wise separable convolutions as efficient building blocks. However, Google says that the 2nd version of MobileNet has two new features:

Linear bottlenecks between the layers: Experimental evidence suggests that using linear layers is crucial as it prevents nonlinearities from destroying too much information. Using non-linear layers in bottlenecks indeed hurts the performance by several percent, further validating our hypothesis
Shortcut connections between the bottlenecks
The Basic Structure of MobileNetV2
The bottlenecks of the MobileNetV2 encode the intermediate inputs and outputs while the inner layer encapsulates the model’s ability to transform from lower-level concepts such as pixels to higher level descriptors such as image categories. With traditional residual connections, shortcuts enable faster training and better accuracy.



Model Architecture
The basic building block is a bottleneck depth-separable convolution with residuals. The architecture of MobileNetV2 contains the initial fully convolution layer with 32 filters, followed by 19 residual bottleneck layers. The researchers have tailored the architecture to different performance points, by using the input image resolution and width multiplier as tunable hyperparameters, that can be adjusted depending on desired accuracy or performance trade-offs. The primary network  (width multiplier 1, 224 × 224), has a computational cost of 300 million multiply-adds and uses 3.4 million parameters. The network computational cost ranges from 7 multiply-adds to 585M MAdds, while the model size varies between 1.7M and 6.9M parameters.

How Is It Different From MobileNetV1?
The MobileNetV2 models are much faster in comparison to MobileNetV1. It uses 2 times fewer operations, has higher accuracy, needs 30 percent fewer parameters and is about 30-40 percent faster on a Google pixel phone.

SEE ALSO
google
Google Announces New AutoML Vision Edge & AutoML Video Upgrades



To enable on-device semantic segmentation, the researcher used MobileNetV2 as a feature extractor in a reduced form of DeepLabv3 that controls the resolution of computed feature maps. On the semantic segmentation benchmark, PASCAL VOC 2012, MobileNetV2 performed similar to MobileNetV1 as feature extractor, but the V2 version requires 5.3 times fewer parameters and 5.2 times fewer operations in terms of multiply-adds.


On A Concluding Note
The new version of MobileNet has several properties that make it suitable for mobile applications and allows very memory-efficient inference and utilises standard operations present in all neural frameworks. For the ImageNet dataset, MobileNetV2 improves the state of the art for a wide range of performance points. For object detection task, it outperforms real-time detectors on COCO datasets. MobileNetV2 provides a very efficient mobile-oriented model that can be used as a base for many visual recognition tasks, claims Google.

https://towardsdatascience.com/review-mobilenetv2-light-weight-model-image-classification-8febb490e61c

Review: MobileNetV2 — Light Weight Model (Image Classification)
Outperforms MobileNetV1, NASNet, and ShuffleNet V1
Sik-Ho Tsang
Sik-Ho Tsang
Follow
May 19, 2019 · 5 min read





Image for post
MobileNetV2 for Mobile Devices
Inthis story, MobileNetV2, by Google, is briefly reviewed. In the previous version MobileNetV1, Depthwise Separable Convolution is introduced which dramatically reduce the complexity cost and model size of the network, which is suitable to Mobile devices, or any devices with low computational power. In MobileNetV2, a better module is introduced with inverted residual structure. Non-linearities in narrow layers are removed this time. With MobileNetV2 as backbone for feature extraction, state-of-the-art performances are also achieved for object detection and semantic segmentation. This is a paper in 2018 CVPR with more than 200 citations. (Sik-Ho Tsang @ Medium)
Outline
MobileNetV2 Convolutional Blocks
Overall Architecture
Ablation Study
Experimental Results
1. MobileNetV2 Convolutional Blocks
Image for post
1.1. MobileNetV1
In MobileNetV1, there are 2 layers.
The first layer is called a depthwise convolution, it performs lightweight filtering by applying a single convolutional filter per input channel.
The second layer is a 1×1 convolution, called a pointwise convolution, which is responsible for building new features through computing linear combinations of the input channels.
ReLU6 is used here for comparison. (Actually, in MobileNetV1 tech report, I cannot find any hints that they use ReLU6… Maybe we need to check the codes in Github…), i.e. min(max(x, 0), 6) as follows:
Image for post
ReLU6
ReLU6 is used due to its robustness when used with low-precision computation, based on [27] MobileNetV1.
1.2. MobileNetV2
In MobileNetV2, there are two types of blocks. One is residual block with stride of 1. Another one is block with stride of 2 for downsizing.
There are 3 layers for both types of blocks.
This time, the first layer is 1×1 convolution with ReLU6.
The second layer is the depthwise convolution.
The third layer is another 1×1 convolution but without any non-linearity. It is claimed that if ReLU is used again, the deep networks only have the power of a linear classifier on the non-zero volume part of the output domain.
Image for post
And there is an expansion factor t. And t=6 for all main experiments.
If the input got 64 channels, the internal output would get 64×t=64×6=384 channels.
2. Overall Architecture
Image for post
MobileNetV2 Overall Architecture
where t: expansion factor, c: number of output channels, n: repeating number, s: stride. 3×3 kernels are used for spatial convolution.
In typical, the primary network (width multiplier 1, 224×224), has a computational cost of 300 million multiply-adds and uses 3.4 million parameters. (Width multiplier is introduced in MobileNetV1.)
The performance trade offs are further explored, for input resolutions from 96 to 224, and width multipliers of 0.35 to 1.4.
The network computational cost up to 585M MAdds, while the model size vary between 1.7M and 6.9M parameters.
To train the network, 16 GPU is used with batch size of 96.
Image for post
Number of Maximum Channels/Memory in Kb) at Each Spatial Resolution for Different Architecture with 16-bit floats for activation
3. Ablation Study
3.1. Impact of Linear Bottleneck
Image for post
With the removal of ReLU6 at the output of each bottleneck module, accuracy is improved.
3.2. Impact of Shortcut
Image for post
With shortcut between bottlenecks, it outperforms shortcut between expansions and the one without any residual connections.
4. Experimental Results
Image for post
MobileNetV2 for Classification, Detection and Segmentation (From https://ai.googleblog.com/2018/04/mobilenetv2-next-generation-of-on.html)
4.1. ImageNet Classification
Image for post
ImageNet Top-1 Accuracy
MobileNetV2 outperforms MobileNetV1 and ShuffleNet (1.5) with comparable model size and computational cost.
With width multiplier of 1.4, MobileNetV2 (1.4) outperforms ShuffleNet (×2), and NASNet with faster inference time.
Image for post
As shown above, different input resolutions and width multipliers are used. It consistently outperforms MobileNetV1.
4.2. MS COCO Object Detection
Image for post
SSDLite
First, SSDLite is introduced by modifying the regular convolutions in SSD with depthwise separable convolutions (MobileNetV1 one).
SSDLite dramatically reduces both parameter count and computational cost.
Image for post
MS COCO Object Detection
MobileNetV2 + SSDLite achieves competitive accuracy with significantly fewer parameters and smaller computational complexity.
And the inference time is faster than MobileNetV1 one.
Notably, MobileNetV2 + SSDLite is 20× more efficient and 10× smaller while still outperforms YOLOv2 on COCO dataset.
4.3. PASCAL VOC 2012 Semantic Segmentation
Image for post
PASCAL VOC 2012 Validation Set
Here, MobileNetV2 is used as feature extractor for DeepLabv3.
With the disabling of Atrous Spatial Pyramid Pooling (ASPP) as well as Multi-Scale and Flipping (MP), also changing the output stride from 8 to 16, mIOU of 75.32% is obtained, with far low of model size and computational cost.
Reference
[2018 CVPR] [MobileNetV2]
MobileNetV2: Inverted Residuals and Linear Bottlenecks

[5. Modyfikacje](05_00_Modyfikacje.ipynb)<br>
[Spis Treści](https://github.com/DarekGit/FACES_DNN/blob/master/notebooks/Praca_Dyplomowa.ipynb)